In [ ]:
# download and install the requires
!pip install tensorflow==1.5.0 # currently Serket uses tensorflow 1 
!git clone https://github.com/naka-lab/Serket.git

     |████████████████████████████████| 44.4MB 91kB/s 
     |████████████████████████████████| 3.0MB 45.4MB/s 
     |████████████████████████████████| 890kB 46.0MB/s 


In [ ]:
# import modules
%cd /content/Serket/
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
import serket as srk
import vae
import gmm
import mlda
import mm
from sklearn.decomposition import PCA
from sklearn.metrics.cluster import adjusted_rand_score

# Example 1

In [ ]:
%cd /content/Serket/examples/VAE-GMM

## Dataset: MNIST

In [ ]:
data = np.loadtxt("data.txt")
labels = np.loadtxt( "category.txt" )

In [ ]:
plt.imshow( data[1].reshape(28,28), cmap="gray" )
print(labels[1])

## Construct and train the integrated model

In [ ]:
# VAE
encoder_dim = 128
decoder_dim = 128

class vae_model(vae.VAE):
    def build_encoder(self, x, latent_dim):
        h_encoder = tf.keras.layers.Dense(encoder_dim, activation="relu")(x)

        mu = tf.keras.layers.Dense(latent_dim)(h_encoder)
        logvar = tf.keras.layers.Dense(latent_dim)(h_encoder)
        
        return mu, logvar
    
    def build_decoder(self, z):
        h_decoder = tf.keras.layers.Dense(decoder_dim, activation="relu")(z)
        logits = tf.keras.layers.Dense(784)(h_decoder)

        optimizer = tf.train.AdamOptimizer()
        
        return logits, optimizer

In [ ]:
# Create instances of the modules
obs = srk.Observation( data )
vae1 = vae_model( 18, itr=200, batch_size=500 )
gmm1 = gmm.GMM( 10 )

# Connect modules
vae1.connect( obs )
gmm1.connect( vae1 )

In [ ]:
# Train the integrated model
for i in range(5):
    print( i )
    vae1.update()
    gmm1.update()

## Evaluation

### Latent space of VAE

In [ ]:
# get the latent variables
feats = vae1.get_forward_msg()

# compress 18 dims to 2 dims by PCA
pca = PCA(n_components=2)
pca.fit(feats)
feats_2dim = pca.transform(feats)

# visualization
for num in range(10):
  f = feats_2dim[num==labels]
  plt.plot( f[:,0], f[:,1], "o", label=str(num) )

plt.legend()
plt.show()  

### Classification accuracy

In [ ]:
# get the probabilities of classificaiton results
preds = gmm1.get_forward_msg()

# convert them to the labels by argmax operation
pred_labels = np.argmax( preds, axis=1 )

# compute the score
print("ARI:", adjusted_rand_score(pred_labels, labels))

# Example 2

In [ ]:
%cd /content/Serket/examples/VAE-GMM-MLDA

## Dataset

In [ ]:
!unzip data.zip

In [ ]:
data1 = np.loadtxt("data/data1.txt")
data2 = np.loadtxt("data/data2.txt")
labels = np.loadtxt( "data/category.txt" )

In [ ]:
# image
plt.imshow( data1[0].reshape(28,28), cmap="gray" )
print(labels[0])

In [ ]:
# speech signal: HAC features (https://www.isca-speech.org/archive/interspeech_2008/i08_2554.html)
plt.bar( range(2400), data2[0] )

## Construct and train the integrated model

In [ ]:
# Create instances of the modules
obs1 = srk.Observation( data1 )
obs2 = srk.Observation( data2 )

vae1 = vae_model( 18, itr=200, batch_size=500 )
gmm1 = gmm.GMM( 10 )
mlda1 = mlda.MLDA( 10, [200,200] )

# Connect modules
vae1.connect( obs1 )
gmm1.connect( vae1 )
mlda1.connect( obs2, gmm1 )

# Train the integrated model
for i in range(5):
    print( i )
    vae1.update()
    gmm1.update()
    mlda1.update()

## Evaluation

### Latent space of VAE

In [ ]:
# get the latent variables
feats = vae1.get_forward_msg()

# compress 18 dims to 2 dims by PCA
pca = PCA(n_components=2)
pca.fit(feats)
feats_2dim = pca.transform(feats)

# visualization
for num in range(10):
  f = feats_2dim[num==labels]
  plt.plot( f[:,0], f[:,1], "o", label=str(num) )

plt.legend()
plt.show()  

### Classification accuracy

In [ ]:
# get the probabilities of classificaiton results
preds = mlda1.get_forward_msg()

# convert them to the labels by argmax operation
pred_labels = np.argmax( preds, axis=1 )

# compute the score
print("ARI:", adjusted_rand_score(pred_labels, labels))

# Example 3

In [ ]:
%cd /content/Serket/examples/VAE-GMM-MLDA-MM

## Dataset

In [ ]:
!unzip data.zip

In [ ]:
data1 = np.loadtxt("data/data1.txt")
data2 = np.loadtxt("data/data2.txt")
labels = np.loadtxt( "data/category.txt" )

In [ ]:
# image
for i in range(20):
  plt.subplot( 2, 10, i+1 )
  plt.imshow( data1[i].reshape(28,28), cmap="gray" )
plt.show()

In [ ]:
# speech signal: HAC features (https://www.isca-speech.org/archive/interspeech_2008/i08_2554.html)
plt.bar( range(2400), data2[0] )

## Construct and train the integrated model

In [ ]:
# Create instances of the modules
obs1 = srk.Observation( data1 )
obs2 = srk.Observation( data2 )

vae1 = vae_model( 18, itr=200, batch_size=500 )
gmm1 = gmm.GMM( 10 )
mlda1 = mlda.MLDA( 10, [200,200] )
mm1 = mm.MarkovModel()

# Connect modules
vae1.connect( obs1 )
gmm1.connect( vae1 )
mlda1.connect( obs2, gmm1 )
mm1.connect( mlda1 )

# Train the integrated model
for i in range(5):
    print( i )
    vae1.update()
    gmm1.update()
    mlda1.update()
    mm1.update()

## Evaluation

### Latent space of VAE

In [ ]:
# get the latent variables
feats = vae1.get_forward_msg()

# compress 18 dims to 2 dims by PCA
pca = PCA(n_components=2)
pca.fit(feats)
feats_2dim = pca.transform(feats)

# visualization
for num in range(10):
  f = feats_2dim[num==labels]
  plt.plot( f[:,0], f[:,1], "o", label=str(num) )

plt.legend()
plt.show()  

### Classification accuracy

In [ ]:
# get the probabilities of classificaiton results
preds = mlda1.get_forward_msg()

# convert them to the labels by argmax operation
pred_labels = np.argmax( preds, axis=1 )

# compute the score
print("ARI:", adjusted_rand_score(pred_labels, labels))